In [1]:
# imports

from mido import Message, MidiFile, MidiTrack
import pandas as pd

import sys
sys.path.append("C:\\Users\\pmkit\\AppData\\Roaming\\Python\\Python313\\site-packages")
from midi2audio import FluidSynth

In [2]:
# grab data we need
pull_data = pd.read_csv("../datasets/WRI363_Pulls_Data.csv")

# if cards are pulled on the same day, we want the music to play all the notes at the same time
notes_list = []
temp_notes_list = []
for i in range(0, pull_data.shape[0] - 1):
    to_add = int(pull_data["Card_Num"][i])
    if pull_data["Date"][i] == pull_data["Date"][i+1]:
        temp_notes_list.append(to_add)
    elif not temp_notes_list:
        notes_list.append(to_add)
    else: # last card in sequence of same dates
        temp_notes_list.append(to_add)
        notes_list.append(temp_notes_list)
        temp_notes_list = []

# add last input
if not temp_notes_list:
    notes_list.append(int(pull_data["Card_Num"][pull_data.shape[0] - 1]))
else:
    temp_notes_list.append(int(pull_data["Card_Num"][pull_data.shape[0] - 1]))
    notes_list.append(temp_notes_list)

print(len(notes_list))
        

116


We now iterate to create file. Nested lists are notes that should all be played simultaneously! 

In [3]:
#pseudocode for now

"""
readjust notes so card 26 is middle C (value 60) -- middle card should be middle C or all the notes will be very low

for each element in notes_list:
    if an int: 
        turn the note on and off
    if a list
        add each note in the list
        turn off each note in the list
"""

'\nreadjust notes so card 26 is middle C (value 60) -- middle card should be middle C or all the notes will be very low\n\nfor each element in notes_list:\n    if an int: \n        turn the note on and off\n    if a list\n        add each note in the list\n        turn off each note in the list\n'

In [4]:
# create MIDI file
def create_file(notes, outputFile, tempo):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    for beat in notes:
        if isinstance(beat, int): # single note
            track.append(Message('note_on', note=beat, velocity=64, time=0))
            track.append(Message('note_off', note=beat, velocity=64, time=tempo))
        else: # chord
            for card in beat:
                track.append(Message('note_on', note=card, velocity=64, time=0))
            for card in beat: 
                track.append(Message('note_off', note=card, velocity=64, time=tempo))

    mid.save(outputFile)

# shift notes to sound higher pitched
def shift_notes(notes, shift):
    for i in range(len(notes)):
        if isinstance(notes[i], int): # single note
            notes[i] += shift
        else: # chord
            for j in range(len(notes[i])):
                notes[i][j] += shift

In [5]:
# testing some files...
create_file(notes_list, "test1.mid", 480)

# readjust notes so card 26 is middle C (value 60) -- middle card should be middle C or all the notes will be very low

shift_notes(notes_list, 34)
create_file(notes_list, "test2.mid", 220)

# taking test2 because I like this one

create_file(notes_list, "richardson_counterdatapiece.mid", 220)


Conversion to .wav was done using Garage Band